In [1]:
#pip install -U git+https://github.com/coin-or/pulp

In [2]:
import pandas as pd
import numpy as np
from pulp import *

df = pd.read_csv('USDA.csv')
df = df.fillna(0)
df.head()

,ID,Description,Calories,Protein,TotalFat,Carbohydrate,Sodium,SaturatedFat,Cholesterol,Sugar,Calcium,Iron,Potassium,VitaminC,VitaminE,VitaminD
0,1001,"BUTTER,WITH SALT",717.0,0.85,81.11,0.06,714.0,51.368,215.0,0.06,24.0,0.02,24.0,0.0,2.32,1.5
1,1002,"BUTTER,WHIPPED,WITH SALT",717.0,0.85,81.11,0.06,827.0,50.489,219.0,0.06,24.0,0.16,26.0,0.0,2.32,1.5
2,1003,"BUTTER OIL,ANHYDROUS",876.0,0.28,99.48,0.00,2.0,61.924,256.0,0.00,4.0,0.00,5.0,0.0,2.80,1.8
3,1004,"CHEESE,BLUE",353.0,21.40,28.74,2.34,1395.0,18.669,75.0,0.50,528.0,0.31,256.0,0.0,0.25,0.5
4,1005,"CHEESE,BRICK",371.0,23.24,29.68,2.79,560.0,18.764,94.0,0.51,674.0,0.43,136.0,0.0,0.26,0.5


In [3]:
data = np.array(df)
data[data == None] = 0

Calories_ = data[:,2].astype(np.float64)
Protein_ = data[:,3].astype(np.float64)
TotalFat_ = data[:,4].astype(np.float64)
Carbohydrate_ = data[:,5].astype(np.float64)
Sodium_  = data[:,6].astype(np.float64)
SaturatedFat_ = data[:,7].astype(np.float64)
Cholesterol_ =  data[:,8].astype(np.float64)
Sugar_ = data[:,9].astype(np.float64)
Calcium_ = data[:,10].astype(np.float64)
Iron_ = data[:,11].astype(np.float64)
Potassium_ = data[:,12].astype(np.float64)
VitaminC_ = data[:,13].astype(np.float64) 
VitaminE_ =  data[:,14].astype(np.float64)
VitaminD_  = data[:,15].astype(np.float64)


cost = np.random.uniform(100, size=(data.shape[0],1))

In [4]:
food_items = list(df['Description'])

calories = dict(zip(food_items,Calories_))

cholesterol = dict(zip(food_items,Cholesterol_))

fat = dict(zip(food_items,TotalFat_))

sodium = dict(zip(food_items,Sodium_))

carbs = dict(zip(food_items,Carbohydrate_))

protein = dict(zip(food_items,Protein_))

vit_C = dict(zip(food_items,VitaminC_))

calcium = dict(zip(food_items,Calcium_))

iron = dict(zip(food_items,Iron_))

vit_E = dict(zip(food_items,VitaminE_))

Sugar = dict(zip(food_items,Sugar_))

costs = dict(zip(food_items,cost))

In [5]:
food_vars = LpVariable.dicts("Food name :",food_items,0,cat='Continuous')

In [6]:
prob = LpProblem("Simple Diet Problem",LpMaximize)

C:\Anaconda3\envs\program\lib\site-packages\pulp\pulp.py:1355: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [7]:



# The objective function is added to 'prob' first
prob += lpSum([vit_C[i]*food_vars[i] for i in food_items]), "Total Cost of the balanced diet"

In [8]:
prob += lpSum([calories[f] * food_vars[f] for f in food_items]) >= 20, "CalorieMinimum"
prob += lpSum([calories[f] * food_vars[f] for f in food_items]) <= 100, "CalorieMaximum"


# Fat
prob += lpSum([fat[f] * food_vars[f] for f in food_items]) >= 600, "FatMinimum"
prob += lpSum([fat[f] * food_vars[f] for f in food_items]) <= 500, "FatMaximum"

# Carbs
prob += lpSum([carbs[f] * food_vars[f] for f in food_items]) >= np.min(Carbohydrate_), "CarbsMinimum"
prob += lpSum([carbs[f] * food_vars[f] for f in food_items]) <= np.mean(Carbohydrate_), "CarbsMaximum"

# sugar
prob += lpSum([Sugar[f] * food_vars[f] for f in food_items]) >= np.min(Sugar_), "SugarMinimum"
prob += lpSum([Sugar[f] * food_vars[f] for f in food_items]) <= np.mean(Sugar_)-np.std(Sugar_), "SugarMaximum"

#Protein
prob += lpSum([protein[f] * food_vars[f] for f in food_items]) >= np.min(Protein_), "ProteinMinimum"
prob += lpSum([protein[f] * food_vars[f] for f in food_items]) <= np.mean(Protein_), "ProteinMaximum"

In [9]:
prob.writeLP("SimpleDietProblem.lp")
print(1)

1


In [10]:
prob.solve()

-1

In [11]:
print("Status:", LpStatus[prob.status])

Status: Infeasible


In [12]:
print("Therefore, the optimal (least cost) balanced diet consists of\n"+"-"*110)

food_data = []
c_Value = []
for v in prob.variables():
    if bool(v.varValue) != 0:
        if v.varValue > 0:
            print(v.name, "=", v.varValue)
            food_data += [v.name]
            c_Value += [v.varValue]
            

Therefore, the optimal (least cost) balanced diet consists of
--------------------------------------------------------------------------------------------------------------
Food_name_:_COFFEE_SUB,CRL_GRAIN_BEV,PDR,PREP_W__WHL_MILK = 3.5480937


In [13]:
food_data = np.array(food_data)

In [14]:
idx = np.argsort(c_Value)[::-1]
sorted_data = food_data[idx]
sorted_data[1:4]

array([], dtype='<U57')

In [15]:
c_Value_sort = np.sort(c_Value)
c_Value_sort 

array([3.5480937])

In [16]:
c_Value_sort.shape

(1,)

In [17]:
prob = LpProblem("Simple Diet Problem",LpMinimize)

prob += lpSum([costs[i]*food_vars[i] for i in food_items]), "Total Cost of the balanced diet"

prob += lpSum([calories[f] * food_vars[f] for f in food_items]) >= 20, "CalorieMinimum"
prob += lpSum([calories[f] * food_vars[f] for f in food_items]) <= 100, "CalorieMaximum"


# Fat
prob += lpSum([fat[f] * food_vars[f] for f in food_items]) >= 600, "FatMinimum"
prob += lpSum([fat[f] * food_vars[f] for f in food_items]) <= 500, "FatMaximum"

# Carbs
prob += lpSum([carbs[f] * food_vars[f] for f in food_items]) >= np.min(Carbohydrate_), "CarbsMinimum"
prob += lpSum([carbs[f] * food_vars[f] for f in food_items]) <= np.mean(Carbohydrate_), "CarbsMaximum"

# sugar
prob += lpSum([Sugar[f] * food_vars[f] for f in food_items]) >= np.min(Sugar_), "SugarMinimum"
prob += lpSum([Sugar[f] * food_vars[f] for f in food_items]) <= np.mean(Sugar_)-np.std(Sugar_), "SugarMaximum"

#Protein
prob += lpSum([protein[f] * food_vars[f] for f in food_items]) >= np.min(Protein_), "ProteinMinimum"
prob += lpSum([protein[f] * food_vars[f] for f in food_items]) <= np.mean(Protein_), "ProteinMaximum"
prob.writeLP("SimpleDietProblem.lp")
print(1)
prob.solve()
print("Status:", LpStatus[prob.status])
print("Therefore, the optimal (least cost) balanced diet consists of\n"+"-"*110)

food_data = []
c_Value = []
for v in prob.variables():
    if bool(v.varValue) != 0:
        if v.varValue > 0:
            print(v.name, "=", v.varValue)
            food_data += [v.name]
            c_Value += [v.varValue]

1
Status: Infeasible
Therefore, the optimal (least cost) balanced diet consists of
--------------------------------------------------------------------------------------------------------------
Food_name_:_FAT,GOOSE = 6.012024
